# Multi-effect distillation (MED) model development
Alternative using GPy (**Gaussian Process Regresion**) instead of MATLAB's ANNs toolbox

In [1]:
from pathlib import Path
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import GPy

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

%load_ext autoreload

# Paths
data_path: Path = Path("../../data")

# Constants
input_ids: list[str] = ["Ms", "Ts_in", "Mf", "Tc_out", "Tc_in"]
output_ids: list[str] = ["Md", "Ts_out", "Mc"]
seed: int = 42


In [2]:
# Load data
with open(data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)


## Calibrate model

In [ ]:
data: pd.DataFrame = pd.read_csv(data_path / 'datasets/operation_points_v2.csv')

display(data)
print(f"{len(data)}")
print(data.columns)


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,2012-04-17 10:47:45+00:00,41.106490,7.787207,1.221267,13.081017,71.829864,67.794467,64.336511,21.368971,44.359224,...,260.556974,229.314313,151.555567,121.170949,95.896905,77.206217,60.501202,45.130574,29.528484,3.30624
841,2012-04-17 10:55:45+00:00,41.104306,7.779161,1.185197,14.131246,71.698753,67.706546,64.280832,21.524272,44.407043,...,259.704036,228.774104,151.542921,121.234136,95.983937,77.289469,60.542123,45.175485,29.530594,3.30624
842,2012-04-17 11:03:45+00:00,41.107775,7.775968,1.201441,15.624742,71.801699,67.759812,64.263024,22.195718,44.246575,...,260.430011,229.121829,151.671249,121.274380,96.003379,77.269748,60.455831,45.009730,29.420515,3.30624
843,2012-04-17 11:11:45+00:00,41.104519,7.766992,1.226294,17.038622,71.894362,67.896687,64.357044,22.605675,44.381693,...,261.433354,229.991126,152.021154,121.638535,96.245695,77.451516,60.582871,45.148548,29.430709,3.30624


845
Index(['time', 'Ms', 'Mf', 'Md', 'Mc', 'Ts_in', 'Ts_out', 'Tf', 'Td', 'Pvc',
       'Tc_in', 'Tc_out', 'Pv_1', 'Pv_2', 'Pv_4', 'Pv_6', 'Pv_8', 'Pv_10',
       'Pv_12', 'Pv_14', 'Tph_ref_14', 'wf'],
      dtype='object')


In [3]:
# Filter operation points
"""
Points are considered different and thus valid if:
- ΔTs,in > 0.5 or
- ΔTs,out > 0.5 or
- ΔTvc > 1.5 or
- ΔMs > 0.2*3.6
- ΔMf > 0.5
- ΔTc,out > 0.5
"""

filtered_data = data.copy()
init_len = 0
while init_len != len(filtered_data):
    init_len = len(filtered_data)
    
    # Filter points based on the given conditions
    filtered_data = data[
        (data['Ts_in'].diff().abs() > 0.5) |
        (data['Ts_out'].diff().abs() > 0.5) |
        (data['Tc_out'].diff().abs() > 1.5) |
        (data['Ms'].diff().abs() / 3.6 > 0.2) |
        (data['Mf'].diff().abs() > 0.5) |
        (data['Tc_out'].diff().abs() > 0.5)
    ]
    
    # Filter points out of range
    filtered_data = data[
        (data['Ts_in'].between(52, 75)) &
        (data['Tc_out'].between(18, 40)) &
        (data['Tc_in'].between(12, 33.8)) &
        (data['Md'] >= 1.8) &
        (data['Mf'].between(5, 9))
    ]
    

display(filtered_data)
print(f"{len(filtered_data)}")

data = filtered_data


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,2011-04-20 10:38:07+00:00,25.891146,8.054595,2.151164,12.729419,64.985897,59.928805,57.778854,29.072973,41.612805,...,183.109206,169.437085,124.743173,102.601095,82.401592,68.075981,54.778751,42.248786,28.606810,3.30624
581,2011-04-26 10:51:07+00:00,25.858430,6.018845,2.085503,8.806182,64.881452,59.956741,57.867972,28.653212,41.576920,...,181.420278,168.891486,123.194795,100.732394,81.360940,67.316296,54.335470,42.227161,28.702012,3.30624
582,2011-04-26 11:01:07+00:00,25.852777,6.012166,2.104099,9.668030,64.894876,59.961908,57.977237,29.030250,41.580232,...,182.536682,169.647905,124.602333,102.100709,82.491204,68.193053,54.777165,42.220583,28.667232,3.30624
583,2011-04-26 11:11:07+00:00,25.820407,6.007560,2.103695,10.414113,64.638266,59.876821,57.877583,29.206987,41.586318,...,181.847411,168.846433,124.474040,102.208168,82.612324,68.324115,54.869832,42.208215,28.651897,3.30624


480


In [4]:
# Divide data between calibration / training and validation
X = data[input_ids].values
Y = data[output_ids].values

# Shuffle the indices
indices = np.arange(len(X))
np.random.seed(seed)  # For reproducibility
np.random.shuffle(indices)

# Define split point
split_index = int(0.8 * len(X))  # 80% for training, 20% for validation

# Split data
train_indices = indices[:split_index]
val_indices = indices[split_index:]

X_train, Y_train = X[train_indices], Y[train_indices]
X_val, Y_val = X[val_indices], Y[val_indices]

print(f"{X.shape=}, {Y.shape=}")
print(f"{X_train.shape=}, {Y_train.shape=}")
print(f"{X_val.shape=}, {Y_val.shape=}")

# output: (n pts, len(input_ids), 2)
np.save(data_path / "med_gpr_data.npy",
        np.stack((X_train, np.pad(Y_train, ( (0, 0), (0, len(input_ids)-len(output_ids)) ))), axis=-1))


X.shape=(480, 5), Y.shape=(480, 3)
X_train.shape=(384, 5), Y_train.shape=(384, 3)
X_val.shape=(96, 5), Y_val.shape=(96, 3)


In [16]:
# Calibrate a model
kernel = GPy.kern.RBF(input_dim=X_train.shape[1], variance=1.0, lengthscale=1.0)  # Radial Basis Function kernel

Y_preds = []
Y_pred_vars = []
models = []
for i in range(Y_train.shape[1]):
    model = GPy.models.GPRegression(X_train, Y_train[:, [i]], kernel.copy())
    model.optimize(messages=True)  # This will optimize the hyperparameters
    models.append(model)
    Y_pred, Y_pred_var = model.predict(X_val)
    Y_preds.append(Y_pred)
    Y_pred_vars.append(Y_pred_var)
Y_preds = np.hstack(Y_preds)  # Combine predictions
Y_pred_vars = np.hstack(Y_pred_vars)  # Combine variances

# model = GPy.models.GPRegression(X_train, Y_train, kernel)
# model.optimize(messages=True)  # This will optimize the hyperparameters



In [18]:
def import_train_data(model_data_path: Path, ydim: int = 3) -> tuple[np.ndarray[float], np.ndarray[float]]:
    """ Import GPR model train data """
    
    gpr_data: np.ndarray[float] = np.load(model_data_path)

    # Split the combined array back into X_train and Y_train (removing the padding)
    X_train = gpr_data[:, :, 0]
    Y_train = gpr_data[:, :, 1][:, :ydim]
    
    return X_train, Y_train
    

class MedModel:
    """
    
    Outputs: qd, Ts,out, qc
    Inputs: Ms, Ts,in, Mf, Tc,out, Tc,in
    
    """
    
    X: np.ndarray[float]
    Y: np.ndarray[float]
    param_array: list[tuple[float, float, float]]
    models: list[GPy.models.GPRegression]
    
    param_array: list[tuple[float, float, float]] = [
        (2.03122253e+00, 5.98578573e+00, 4.47722101e-03),
        (1.81219256e+03, 1.03129028e+02, 1.47705700e-02),
        (133.14646561,   4.93363467,   0.29595623)
        ]
    
    def __init__(self, X: np.ndarray[float] = None, Y: np.ndarray[float] = None, train_data_path: Path = None, 
                 param_array: list[tuple[float, float, float]] = None):
        
        assert (X is not None and Y is not None) or train_data_path is not None, \
            "Either train data (`X` and `Y`) are provided, or a valid (`train_data_path`) path to load them using `import_model_data` needs to be provided"
        
        if train_data_path is not None:
            self.X, self.Y = import_train_data(train_data_path)
        else:
            self.X = X
            self.Y = Y
        
        if param_array is not None:
            self.param_array = param_array
        self.models = self._create_models()
    
    def _create_models(self) -> list[GPy.models.GPRegression]:
        models = []
        for i, param in enumerate(self.param_array):
            model = GPy.models.GPRegression(self.X, self.Y[:, [i]], initialize=False)
            model.update_model(False) # do not call the underlying expensive algebra on load
            model.initialize_parameter() # Initialize the parameters (connect the parameters up)
            model[:] = param # Load the parameters
            model.update_model(True) # Call the algebra only once
            models.append(model)
        return models
    
    def predict(self, X: np.ndarray, return_variances: bool = False) -> np.ndarray[float] | tuple[np.ndarray[float], np.ndarray[float]]:
        
        Y_preds, Y_pred_vars = zip(*(model.predict(X) for model in self.models))
        Y_preds = np.hstack(Y_preds)  # Combine predictions
        
        if return_variances:
            return Y_preds, np.hstack(Y_pred_vars)  # Combine variances
        return Y_preds


# Both options should work
med_model = MedModel(*import_train_data(data_path / "med_gpr_data.npy")) # Provide X and Y
med_model = MedModel(train_data_path=data_path / "med_gpr_data.npy") # Provide path to load X and Y

# Evaluate validation dataset
Y_preds, Y_pred_vars = med_model.predict(X_val, return_variances=True)

print(f"{X_val.shape=}")
print(f"{Y_preds[:, 0].shape=}")
print(f"{Y_preds.shape=}")
print(f"{Y_pred_vars.shape=}")


 /workspaces/solarMED-modeling/.venv/lib/python3.11/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


X_val.shape=(96, 5)
Y_preds[:, 0].shape=(96,)
Y_preds.shape=(96, 3)
Y_pred_vars.shape=(96, 3)


In [21]:
# Visualize fit

# Create traces for the plot
fig = make_subplots(rows=len(output_ids), cols=1, shared_xaxes=True, 
                    subplot_titles=output_ids, x_title="Data Index",
                    vertical_spacing=0.05)

for idx, output_id in enumerate(output_ids):
    # Add true values as a scatter plot
    fig.add_trace(go.Scatter(
        x=np.arange(Y_val.shape[0]),
        y=Y_val[:, idx],
        mode='markers',
        name='True values',
        showlegend=True if idx==0 else False,
        marker=dict(color='blue')
    ), row=idx+1, col=1)

    # Add predicted values as a line plot
    fig.add_trace(go.Scatter(
        x=np.arange(Y_preds[:, idx].shape[0]),
        y=Y_preds[:, idx],
        mode='lines',
        name='Predicted values',
        showlegend=True if idx==0 else False,
        line=dict(color='red')
    ), row=idx+1, col=1)

    # Add confidence interval as a shaded area
    fig.add_trace(go.Scatter(
        x=np.concatenate([np.arange(Y_preds[:,idx].shape[0]), np.arange(Y_preds[:,idx].shape[0])[::-1]]),
        y=np.concatenate([Y_preds[:, idx] - 2 * np.sqrt(Y_pred_vars[:, idx]), 
                        (Y_preds[:, idx] + 2 * np.sqrt(Y_pred_vars[:, idx]))[::-1]]),
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.2)',
        line=dict(color='rgba(255, 0, 0, 0)'),
        name='Confidence interval (95%)',
        showlegend=True if idx==0 else False,
    ), row=idx+1, col=1)

# Update layout
fig.update_layout(
    title="Regression Results",
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="right", x=1),
    template="plotly_white",
    height=600,
    margin=dict(t=100)
)

# Show the plot
fig.show()

# Visualization 2. 
# Create a figure
fig = go.Figure()

# Iterate over each output variable
for i, output_name in enumerate(output_ids):
    # Calculate correlation coefficient
    correlation = np.corrcoef(Y_val[:, i], Y_preds[:, i])[0, 1]
    
    # Add scatter plot for true vs. predicted
    fig.add_trace(go.Scatter(
        x=Y_val[:, i],  # True outputs for the i-th variable
        y=Y_preds[:, i],  # Predicted outputs for the i-th variable
        mode='markers',
        name=f'{output_name} (r={correlation:.2f})',
        marker=dict(size=6, opacity=0.7),
        showlegend=True
    ))

    # Add perfect correlation line (y=x)
    max_val = max(Y_val[:, i].max(), Y_preds[:, i].max())
    min_val = min(Y_val[:, i].min(), Y_preds[:, i].min())
    fig.add_trace(go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        name=f'{output_name} Perfect Correlation (y=x)',
        line=dict(color='black', dash='dash'),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title="True vs Predicted Outputs with Correlation Coefficients",
    xaxis_title="True Outputs",
    yaxis_title="Predicted Outputs",
    legend=dict(yanchor="top", y=0.3, xanchor="left", x=0.7),
    template="plotly_white"
)

# Show the plot
fig.show()



## Evaluate model


In [3]:
# Evaluate some experimental timeseries
from solarmed_modeling.data_validation import within_range_or_nan_or_max, within_range_or_zero_or_max
from solarmed_modeling.utils import data_preprocessing, data_conditioning

from solarmed_modeling.med import MedModel

logger.disable("phd_visualizations.utils.units")

date_str: str = 20230511 # "20230511"  # '20230630'
sample_rate: int = 400

# Load data and preprocess data
df = data_preprocessing([data_path / f"datasets/{fn}" for fn in [f"{date_str}_solarMED.csv", f"{date_str}_MED.csv"]], 
                        vars_config, sample_rate_key=f"{sample_rate}s")
# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate, vars_config=vars_config,)

df


 /workspaces/SolarMED/modeling/src/solarmed_modeling/utils/__init__.py:138: DtypeWarning:Columns (42,50,66,73,75) have mixed types. Specify dtype option on import or set low_memory=False.
 /workspaces/SolarMED/modeling/src/solarmed_modeling/utils/__init__.py:167: FutureWarning:Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
2025-08-30 11:47:38.887 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id PK-MED-E03-pa not found in dataframe columns.
2025-08-30 11:47:38.888 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:129 - Signal id FT-SF-003 not found in dataframe columns.


,qmed_s,qts_dis_original,Tts_h_out,Tmed_s_in,Tts_c_in,ZT-AQU-TCV102,wmed_f,qmed_f,qmed_d,Tmed_c_out,...,Psf_l2,Psf_l3,Psf_l4,Psf_l5,Pth_ts_src,Pth_hx_s,Pth_ts_dis,sf_active,med_active,ts_active
time,,,,,,,,,,,,,,,,,,,,,
2023-05-11 07:26:40+00:00,0.0,0.033446,75.749327,49.095691,43.006526,0.593134,103.759155,0.0,0.0,18.195833,...,1.236760,0.672183,3.570127,2.223859,0.0,0.0,0.129382,True,False,False
2023-05-11 07:33:20+00:00,0.0,0.033472,75.527441,49.057274,42.932932,0.599137,103.760875,0.0,0.0,18.318358,...,3.338351,3.068519,6.019116,4.634292,0.0,0.0,0.129298,True,False,False
2023-05-11 07:40:00+00:00,0.0,0.033436,75.516380,48.817047,42.824880,0.604186,103.762608,0.0,0.0,18.494865,...,3.783896,3.747119,7.010392,5.573010,0.0,0.0,0.126668,True,False,False
2023-05-11 07:46:40+00:00,0.0,0.033453,75.353377,48.603423,42.722086,0.605678,103.764347,0.0,0.0,18.684251,...,3.011230,3.189443,6.601975,4.692048,0.0,0.0,0.123942,True,False,False
2023-05-11 07:53:20+00:00,0.0,0.033479,75.421423,48.419229,42.629829,0.608455,103.765625,0.0,0.0,18.801903,...,2.997304,2.993600,6.414546,4.199520,0.0,0.0,0.122860,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-11 16:46:40+00:00,0.0,0.032713,48.805451,49.033724,55.275760,99.971269,8.061619,0.0,0.0,26.493845,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True
2023-05-11 16:53:20+00:00,0.0,0.032705,48.480917,48.964636,54.926018,99.967698,8.249758,0.0,0.0,26.359626,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True
2023-05-11 17:00:00+00:00,0.0,0.032902,48.183382,48.857430,54.576655,99.965503,8.213361,0.0,0.0,26.233246,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True


In [4]:
idx_start = 10
idx_end = len(df)
df_mod = pd.DataFrame()

med_model = MedModel()

# Run model
for idx in range(idx_start,idx_end):
    ds = df.iloc[idx]
        
    start_time = time.time()

    Tmed_c_out = ds["Tmed_c_out"]
    Tmed_c_in = ds["Tmed_c_in"]
    mmed_f = ds["qmed_f"]
    mmed_f = within_range_or_nan_or_max(mmed_f, range=(5, 9), var_name="qmed_f")
    mmed_s = within_range_or_zero_or_max(ds["qmed_s"], range=(30, 48), var_name="qmed_s")
    Tmed_s_in = within_range_or_nan_or_max(ds["Tmed_s_in"], range=(60, 75), var_name="Tmed_s_in")

    med_model_solved = False
    out = {output_id: np.nan for output_id in output_ids}
    while not med_model_solved and (Tmed_c_in < Tmed_c_out < 40) and mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:   
    
    # input_ids: list[str] = ["Ms", "Ts_in", "Mf", "Tc_out", "Tc_in"]
    # output_ids: list[str] = ["Md", "Ts_out", "Mc"]
    
    # if mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:
        inputs = np.array([[
            mmed_s,      # m³/h
            Tmed_s_in,   # ºC
            mmed_f,      # m³/h
            Tmed_c_out,  # ºC
            Tmed_c_in    # ºC
        ]])
        
        # for output_id, model in zip(output_ids, models):
        y_pred = med_model.predict(inputs)
        out[output_ids[0]], out[output_ids[1]], out[output_ids[2]]  = y_pred[0, 0], y_pred[0, 1], y_pred[0, 2]

        if out["Mc"] > 23:
            Tmed_c_out = Tmed_c_out + 1
        elif out["Mc"] < 9:
            Tmed_c_out = Tmed_c_out - 1
        else:
            med_model_solved = True
                  
    logger.info(f"Iteration {idx} / {len(df)} | Elapsed time: {time.time()-start_time:.2f} seconds")
    
    result = pd.DataFrame(
        {
            "qmed_f": mmed_f,
            "qmed_d": out["Md"],
            "qmed_c": out["Mc"],
            "qmed_s": mmed_s,
            "Tmed_s_in": Tmed_s_in,
            "Tmed_s_out": out["Ts_out"], 
            "Tmed_c_in": ds["Tmed_c_in"],
            "Tmed_c_out": Tmed_c_out,
        },
        index=[0]
    )
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)


2025-08-30 11:47:39.607 | INFO     | __main__:<module>:47 - Iteration 10 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.610 | INFO     | __main__:<module>:47 - Iteration 11 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.612 | INFO     | __main__:<module>:47 - Iteration 12 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.614 | INFO     | __main__:<module>:47 - Iteration 13 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.616 | INFO     | __main__:<module>:47 - Iteration 14 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.617 | INFO     | __main__:<module>:47 - Iteration 15 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.619 | INFO     | __main__:<module>:47 - Iteration 16 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.620 | INFO     | __main__:<module>:47 - Iteration 17 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.622 | INFO     | __main__:<module>:47 - Iteration 18 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.624 | INFO     | 

2025-08-30 11:47:39.629 | INFO     | __main__:<module>:47 - Iteration 20 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.632 | INFO     | __main__:<module>:47 - Iteration 21 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.636 | INFO     | __main__:<module>:47 - Iteration 22 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.639 | INFO     | __main__:<module>:47 - Iteration 23 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.642 | INFO     | __main__:<module>:47 - Iteration 24 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.632 | INFO     | __main__:<module>:47 - Iteration 21 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.636 | INFO     | __main__:<module>:47 - Iteration 22 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.639 | INFO     | __main__:<module>:47 - Iteration 23 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.642 | INFO     | __main__:<module>:47 - Iteration 24 / 89 | Elapsed time: 0.00 seconds
2025-08-30 11:47:39.645 | INFO     | 

In [5]:
df_mod


,qmed_f,qmed_d,qmed_c,qmed_s,Tmed_s_in,Tmed_s_out,Tmed_c_in,Tmed_c_out
0,NaN,NaN,NaN,0.0,NaN,NaN,19.733000,19.389720
1,NaN,NaN,NaN,0.0,NaN,NaN,19.562750,19.122885
2,NaN,NaN,NaN,0.0,NaN,NaN,19.370750,18.880558
3,NaN,NaN,NaN,0.0,NaN,NaN,19.225500,18.699255
4,NaN,NaN,NaN,0.0,NaN,NaN,19.050750,18.392908
...,...,...,...,...,...,...,...,...
74,NaN,NaN,NaN,0.0,NaN,NaN,26.129500,26.493845
75,NaN,NaN,NaN,0.0,NaN,NaN,26.024750,26.359626
76,NaN,NaN,NaN,0.0,NaN,NaN,25.911750,26.233246
77,NaN,NaN,NaN,0.0,NaN,NaN,25.782000,26.053022


In [11]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

with open(data_path / 'plt_config_med.json') as f:
    plt_config = json.load(f)
    
fig = experimental_results_plot(
    plt_config, 
    df, 
    df_comp=df_mod, 
    vars_config=vars_config, 
    resample=False,
    comp_trace_labels=[f"[Ts={sample_rate}s]"]
)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'med_validation_{df.index[0].strftime("%Y%m%d")}')
)


### Evaluate and generate visualizations for model with different sample rates and for many dates

(This cell is self-contained and can be run without running previous cells.)

In [1]:
from pathlib import Path
from solarmed_modeling.med import FixedModelParameters
from solarmed_modeling.med.benchmark import benchmark_model

data_path: Path = Path("../../data")
results_path: Path = Path("../../results/models_validation")

stats = benchmark_model(
    fixed_model_params=FixedModelParameters(),
    data_path=data_path,
    output_path=results_path,
    sample_rates=[5, 400],
    save_results=True
)


Processing test 20230629 (1/11)


 /workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:divide by zero encountered in divide
 /workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:divide by zero encountered in divide
2025-08-30 12:31:33.180 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230629.png
2025-08-30 12:31:33.274 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230629.html
2025-08-30 12:31:33.841 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230629.png
2025-08-30 12:31:33.884 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230629.html
2025-08-30 12:31:34.349 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20230629.png


Processing test 20230703 (2/11)


/workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:

divide by zero encountered in divide

/workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:

divide by zero encountered in divide

2025-08-30 12:31:43.862 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230703.png
2025-08-30 12:31:44.419 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230703.html
2025-08-30 12:31:46.267 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230703.png
2025-08-30 12:31:46.309 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230703.html
2025-08-30 12:31:47.980 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20230703.

Processing test 20230511 (3/11)


...

/workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:

divide by zero encountered in divide

/workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:

divide by zero encountered in divide

2025-08-30 12:32:32.720 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20231031.png
2025-08-30 12:32:32.810 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20231031.html
2025-08-30 12:32:33.327 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20231031.png
2025-08-30 12:32:33.353 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20231031.html
2025-08-30 12:32:33.830 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20231031.

Processing test 20230707_20230710 (10/11)


/workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:

divide by zero encountered in divide

2025-08-30 12:34:10.125 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230707.png
2025-08-30 12:34:10.514 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230707.html
2025-08-30 12:34:12.327 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230707.png
2025-08-30 12:34:12.360 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230707.html
2025-08-30 12:34:14.090 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20230707.png
2025-08-30 12:34:14.121 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validatio

Processing test 20230628 (11/11)


2025-08-30 12:34:21.344 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230628.png
2025-08-30 12:34:21.721 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_validation_20230628.html
2025-08-30 12:34:23.352 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230628.png
2025-08-30 12:34:23.381 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_5s_20230628.html
2025-08-30 12:34:24.943 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20230628.png
2025-08-30 12:34:24.974 | INFO     | phd_visualizations:save_figure:41 - Figure saved in ../../results/models_validation/med_regression_400s_20230628.html


### Generate stacked figure with calibration and validation tests

(This cell is self-contained and can be run without running previous cells.)

In [ ]:
import pandas as pd
from pathlib import Path
from itertools import product
import hjson
from phd_visualizations.utils import stack_images_horizontally

data_path = Path("../../data")
results_path = Path("../../results/models_validation")

# plot_config = hjson.load(open(data_path / "plt_config_thermal_storage.json"))
# vars_config = hjson.load(open( data_path / "variables_config.hjson"))

stack_images_horizontally(
    [results_path / f"fig{idx+1}.png" for idx in range(2)],
    output_path=results_path / "med_validation.png",
)
